In [ ]:
!pip install datasets
!pip install sentence-transformers
!pip install openai
!pip install transformers

In [1]:
from sentence_transformers import SentenceTransformer

import datasets

In [2]:
medembed_model = SentenceTransformer("abhinand/MedEmbed-small-v0.1")

In [3]:
bge_model = SentenceTransformer("BAAI/bge-small-en-v1.5")

In [4]:
mini_lm = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
data = datasets.load_dataset('xhluca/publichealth-qa', split='test', name='english')

In [6]:
answers = data.to_pandas()['answer'].to_list()

In [7]:
# Create embeddings using MedEmbed model
medembed_answers = medembed_model.encode(answers)

# Create embeddings using BGE model 
bge_answers = bge_model.encode(answers)

mini_lm_answers = mini_lm.encode(answers)


c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [14]:
import numpy as np

# Function to perform RAG using cosine similarity
def get_similar_answers(query, embeddings_model, answer_embeddings, answers, top_k=3):
    # Encode the query
    query_embedding = embeddings_model.encode(query)
    
    # Calculate cosine similarities
    similarities = embeddings_model.similarity(query_embedding.reshape(1,-1), answer_embeddings)
    
    # Get indices of top k similar answers
    top_indices = np.argsort(similarities[0])[-top_k:]
    
    return [(answers[i], similarities[0][i].item()) for i in top_indices]

# Example query
# query = "What is CDC doing about COVID-19?"
query = "How can I protect my child from COVID-19 infection?"

print("Results using MedEmbed model:")
medembed_results = get_similar_answers(query, medembed_model, medembed_answers, answers)
medembed_results.sort(key=lambda x: x[1], reverse=True)
for answer, score in medembed_results:
    print(f"\nScore: {score:.4f}")
    print(f"Answer: {answer}")
    
    
print("\n\n ============================================================== \n\n")

print("Results using MiniLM model:")
mini_lm_results = get_similar_answers(query, mini_lm, mini_lm_answers, answers)
mini_lm_results.sort(key=lambda x: x[1], reverse=True)
for answer, score in mini_lm_results:
    print(f"\nScore: {score:.4f}")
    print(f"Answer: {answer}")

# print("\nResults using BGE model:")
# bge_results = get_similar_answers(query, bge_model, bge_answers, answers)
# bge_results.sort(key=lambda x: x[1], reverse=True)
# for answer, score in bge_results:
#     print(f"\nScore: {score:.4f}")
#     print(f"Answer: {answer}")


Results using MedEmbed model:

Score: 0.8413
Answer: You can encourage your child to help stop the spread of COVID-19 by teaching them to do the same things everyone should do to stay healthy.

Avoid close contact with people who are sick.
Stay home when you are sick, except to get medical care.
Cover your coughs and sneezes with a tissue and throw the tissue in the trash.
Wash your hands often with soap and water for at least 20 seconds, especially after blowing your nose, coughing, or sneezing; going to the bathroom; and before eating or preparing food.
If soap and water are not readily available, use an alcohol-based hand sanitizer with at least 60% alcohol. Always wash hands with soap and water if hands are visibly dirty.
Clean and disinfect frequently touched surfaces and objects (e.g., tables, countertops, light switches, doorknobs, and cabinet handles).
Launder items, including washable plush toys, as appropriate and in accordance with the manufacturer’s instructions. If possibl

In [15]:
from utils import call_openai


# query = "What is the function of the fibrous membrane?"

def build_rag_response(query, context_results, model="gpt-4o"):
    # Prepare context from top results
    context = "\n\n".join([result[0] for result in context_results[:3]])
    
    system_prompt = """You are a helpful medical assistant. Use the provided context to answer the user's question.
If you cannot answer the question based on the context, say so - do not make up information.
Always cite specific parts of the context to support your answer."""

    user_prompt = f"""Context:
{context}

Question:
{query}

Please provide a detailed answer based on the context above."""

    response = call_openai(system_prompt, user_prompt, model=model)
    return response

# Test the RAG system with different embedding models
print("RAG Response using MedEmbed embeddings:")
medembed_rag_response = build_rag_response(query, medembed_results)
print(medembed_rag_response)

print("\n\n ============================================================== \n\n")

# print("RAG Response using MiniLM embeddings:")
# mini_lm_rag_response = build_rag_response(query, mini_lm_results)
# print(mini_lm_rag_response)


RAG Response using MedEmbed embeddings:
To protect your child from COVID-19 infection, you can follow several preventive measures outlined in the context:

1. **Avoid Close Contact**: Keep your child away from people who are sick. This includes avoiding in-person playdates with children from other households and ensuring they maintain a distance of at least 6 feet from anyone not in their household when playing outside.

2. **Stay Home When Sick**: If your child is sick, keep them at home except to get medical care. This helps prevent spreading the virus to others.

3. **Good Hygiene Practices**:
   - Teach your child to cover their coughs and sneezes with a tissue and dispose of the tissue immediately.
   - Encourage frequent handwashing with soap and water for at least 20 seconds, especially after blowing their nose, coughing, sneezing, using the bathroom, and before eating or preparing food. If soap and water are not available, use an alcohol-based hand sanitizer with at least 60% a

In [9]:
# bge_results.sort(key=lambda x: x[1], reverse=True)
# bge_results[:3]

In [10]:
from datasets import load_dataset

ds = load_dataset("MedRAG/textbooks", split='train[:2000]')

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [11]:
content = ds.to_pandas()['content'].to_list()

# bge_content_embeddings = bge_model.encode(content)
medembed_content_embeddings = medembed_model.encode(content)
mini_lm_content_embeddings = mini_lm.encode(content)



In [12]:
query = "How do anchoring microfibrils influence cellular signaling between cementum-producing and bone-producing cells during physiological displacement?"

print("Results using MedEmbed model:")
medembed_results = get_similar_answers(query, medembed_model, medembed_content_embeddings, content)
medembed_results.sort(key=lambda x: x[1], reverse=True)
for answer, score in medembed_results:
    print(f"\nScore: {score:.4f}")
    print(f"Answer: {answer}")
    
    
# print("\n\n ============================================================== \n\n")

# print("\nResults using BGE model:")
# bge_results = get_similar_answers(query, bge_model, bge_content_embeddings, content)
# bge_results.sort(key=lambda x: x[1], reverse=True)
# for answer, score in bge_results:
#     print(f"\nScore: {score:.4f}")
#     print(f"Answer: {answer}")
    
    
print("\n\n ============================================================== \n\n")

print("Results using MiniLM model:")
mini_lm_results = get_similar_answers(query, mini_lm, mini_lm_content_embeddings, content)
mini_lm_results.sort(key=lambda x: x[1], reverse=True)
for answer, score in mini_lm_results:
    print(f"\nScore: {score:.4f}")
    print(f"Answer: {answer}")


Results using MedEmbed model:

Score: 0.7023
Answer: The fibrous membrane is formed by dense connective tissue and surrounds and stabilizes the joint. Parts of the fibrous membrane may thicken to form ligaments, which further stabilize the joint. Ligaments outside the capsule usually provide additional reinforcement. Another common but not universal feature of synovial joints is the presence of additional structures within the area enclosed by the capsule or synovial membrane, such as articular discs (usually composed of fibrocartilage), fat pads, and tendons. Articular discs absorb compression forces, adjust to changes in the contours of joint surfaces during movements, and increase the range of movements that can occur at joints. Fat pads usually occur between the synovial membrane and the capsule and move into and out of regions as joint contours change during movement. Redundant regions of the synovial membrane and fibrous membrane allow for large movements at joints. Descriptions 

In [13]:
from utils import call_openai


query = "What is the function of the fibrous membrane?"

def build_rag_response(query, context_results, model="gpt-4o"):
    # Prepare context from top results
    context = "\n\n".join([result[0] for result in context_results[:3]])
    
    system_prompt = """You are a helpful medical assistant. Use the provided context to answer the user's question.
If you cannot answer the question based on the context, say so - do not make up information.
Always cite specific parts of the context to support your answer."""

    user_prompt = f"""Context:
{context}

Question:
{query}

Please provide a detailed answer based on the context above."""

    response = call_openai(system_prompt, user_prompt, model=model)
    return response

# Test the RAG system with different embedding models
print("RAG Response using MedEmbed embeddings:")
medembed_rag_response = build_rag_response(query, medembed_results)
print(medembed_rag_response)

print("\n\n ============================================================== \n\n")

print("RAG Response using MiniLM embeddings:")
mini_lm_rag_response = build_rag_response(query, mini_lm_results)
print(mini_lm_rag_response)


RAG Response using MedEmbed embeddings:
The fibrous membrane is formed by dense connective tissue and serves to surround and stabilize the joint. Parts of the fibrous membrane may thicken to form ligaments, which provide further stabilization to the joint. Additionally, ligaments located outside the capsule usually offer additional reinforcement. This structure is crucial for maintaining the integrity and function of synovial joints.




RAG Response using MiniLM embeddings:
The fibrous membrane is formed by dense connective tissue and serves to surround and stabilize the joint. It provides structural support and stability to the joint by forming a protective layer around it. Additionally, parts of the fibrous membrane may thicken to form ligaments, which further contribute to the stabilization of the joint. Ligaments outside the capsule offer additional reinforcement, ensuring that the joint remains secure and functions properly during movement.
